<a href="https://colab.research.google.com/github/masnormen/Coursera_Capstone/blob/main/Battle_of_the_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [164]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
!pip install geopandas -q
import geopandas as gpd
import requests
print('Libraries imported.')

Libraries imported.


# Import Surabaya districts data

### Extracting names and coordinates data from .geojson file

We will use the feature marking administration center in the file and then load it into a dataframe.

In [165]:
df_raw = gpd.read_file("https://drive.google.com/uc?export=download&id=1-qyf8HigwyldHQe_Uy6kIAZWzrTbcAYB")

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [166]:
df_raw

,id,@id,admin_level,name,type,boundary,is_in:city,is_in:province,source,geometry
0,relation/8224396,relation/8224396,6,Genteng,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.74735 -7.24452, 112.74737 -7.245..."
1,relation/8224405,relation/8224405,6,Simokerto,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.75665 -7.23447, 112.75636 -7.234..."
2,relation/8224478,relation/8224478,6,Semampir,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.75665 -7.23447, 112.75769 -7.234..."
3,relation/8224558,relation/8224558,6,Kenjeran,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.78130 -7.22517, 112.78132 -7.225..."
4,relation/8224593,relation/8224593,6,Bulak,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.78618 -7.22616, 112.78557 -7.225..."
5,relation/8224636,relation/8224636,6,Krembangan,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.70702 -7.22765, 112.70713 -7.227..."
6,relation/8224671,relation/8224671,6,Bubutan,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.71721 -7.24701, 112.71653 -7.248..."
7,relation/8224735,relation/8224735,6,Sawahan,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.71432 -7.25609, 112.71367 -7.260..."
8,relation/8224825,relation/8224825,6,Sukomanunggal,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.70708 -7.27925, 112.70719 -7.278..."
9,relation/8224843,relation/8224843,6,Asemrowo,boundary,administrative,Surabaya,Jawa Timur,HOT_InAWARESurvey_2016,"POLYGON ((112.70104 -7.25266, 112.70122 -7.252..."


In [167]:
df = []

In [168]:
df_raw['geometry'] = df_raw['geometry'].centroid

for row in df_raw.itertuples():
    # print(row)
    name = row.name
    longitude = row.geometry.x
    latitude = row.geometry.y
    df.append([name, latitude, longitude])

df = pd.DataFrame(df)
df.columns = ["name", "lat", "lon"]
df.lat = df.lat.astype(float)
df.lon = df.lon.astype(float)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


,name,lat,lon
0,Genteng,-7.258943,112.744854
1,Simokerto,-7.239466,112.753292
2,Semampir,-7.213868,112.748414
3,Kenjeran,-7.217600,112.768674
4,Bulak,-7.236022,112.788849
5,Krembangan,-7.228817,112.721646
6,Bubutan,-7.248752,112.727749
7,Sawahan,-7.273047,112.721540
8,Sukomanunggal,-7.268823,112.699726
9,Asemrowo,-7.237168,112.690060


## Map visualization of Surabaya districts

In [170]:
geolocator = Nominatim(user_agent="surabaya_battle")

location = geolocator.geocode("Surabaya", timeout=10)
latitude = location.latitude
longitude = location.longitude
map_out = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(df['lat'], df['lon'], df['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_out)  
    
map_out

# Making use of Foursquare's API

### Fetching venues for each district

In [171]:
CLIENT_ID = 'QOR5CN0SPNGEMRAM0HXKICOW5X3SYHSCTOIULJNQJEXGJPIV' # your Foursquare ID
CLIENT_SECRET = 'XSEH1J1OOEDWCYM4MBYKKKI00FFJ2A5ZHKKMQDUUASTPM04Y' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 100

print('CLIENT_ID:', CLIENT_ID)
print('CLIENT_SECRET:', CLIENT_SECRET)

CLIENT_ID: QOR5CN0SPNGEMRAM0HXKICOW5X3SYHSCTOIULJNQJEXGJPIV
CLIENT_SECRET: XSEH1J1OOEDWCYM4MBYKKKI00FFJ2A5ZHKKMQDUUASTPM04Y


We will then make a function to get venues around a point (latitude and longitude) within a defined radius. We will get only food-related venues. Notice the `&section=food` part in the API URL.

In [172]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=food'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

Surabaya's district centers are so spread. Some is in the downtown where it's densely populated but some is in the middle of nowhere. So, we will set a radius of 2000 meters to get all food in each district. And then put it into a DataFrame we can work later.

In [173]:
venues = getNearbyVenues(
    names=df['name'],
    latitudes=df['lat'],
    longitudes=df['lon'],
    radius=2000
)

In [174]:
venues.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Genteng,-7.258943,112.744854,Soto Ayam Cak To,-7.254999,112.744358,Soup Place
1,Genteng,-7.258943,112.744854,Rumah Makan Padang Sari Bundo Sati,-7.260683,112.745682,Padangnese Restaurant
2,Genteng,-7.258943,112.744854,Depot Soto Banjar Ahmad Jais,-7.255516,112.742352,Indonesian Restaurant
3,Genteng,-7.258943,112.744854,Soto Ayam Ambengan Pak Sadi Asli,-7.255842,112.745012,Soup Place
4,Genteng,-7.258943,112.744854,Pos Ketan Legenda - 1967,-7.262959,112.741330,Snack Place


In [175]:
venues.shape

(1815, 7)

### Removing overlapping venue

Notice that because we set the radius to 2000 meters, some venues might be listed twice in two/more different districts, which can create duplicates. We will only keep the venues with the closest distance to a district.

In [176]:
venues_dup = venues[venues.duplicated(subset=["Venue", "Venue Latitude", "Venue Longitude", "Venue Category"], keep=False)]
venues = venues.drop(venues_dup.index, axis=0)

In [177]:
# A function to calculate distance between two longitude/latitude points

def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

venues_dup["distance"] = haversine(venues_dup["District Latitude"], venues_dup["District Longitude"], venues_dup["Venue Latitude"], venues_dup["Venue Longitude"])

In [178]:
venues_dup.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,distance
0,Genteng,-7.258943,112.744854,Soto Ayam Cak To,-7.254999,112.744358,Soup Place,0.441906
1,Genteng,-7.258943,112.744854,Rumah Makan Padang Sari Bundo Sati,-7.260683,112.745682,Padangnese Restaurant,0.213981
2,Genteng,-7.258943,112.744854,Depot Soto Banjar Ahmad Jais,-7.255516,112.742352,Indonesian Restaurant,0.470425
4,Genteng,-7.258943,112.744854,Pos Ketan Legenda - 1967,-7.262959,112.741330,Snack Place,0.591996
5,Genteng,-7.258943,112.744854,Yoshinoya,-7.263024,112.739197,Japanese Restaurant,0.771433


Then, we can sort the venues based on the distance in an ascending order. We will then drop the duplicates which will keep only the first item (which is the venue with the shortest distance to a district's center).

And then append to the original venues dataframe.

In [179]:
venues_dup = venues_dup.sort_values(["Venue", "Venue Latitude", "Venue Longitude", "distance"], ascending=True)
venues_dup = venues_dup.drop_duplicates(["Venue", "Venue Latitude", "Venue Longitude"], keep="first").reset_index(drop=True)
venues_dup = venues_dup.drop(columns=['distance'])

In [180]:
venues = venues.append(venues_dup).reset_index(drop=True)

In [181]:
venues.shape

(1251, 7)

# Data exploration

### Visualization of food venues spread

In [182]:
map_out = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(venues['Venue Latitude'], venues['Venue Longitude'], venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_out)  
    
map_out

In [206]:
venues[["District", 'Venue Latitude', 'Venue Longitude', 'Venue']].groupby("District").count()["Venue"].sort_values(ascending=False).to_frame()

,Venue
District,
Mulyorejo,98
Wonokromo,84
Gubeng,84
Dukuh Pakis,79
Tambaksari,75
Gayungan,75
Tenggilis Mejoyo,67
Genteng,64
Simokerto,63


In [184]:
# Lowest number of venues

venues_count_rank = venues[["District", 'Venue Latitude', 'Venue Longitude', 'Venue']].groupby("District").count()["Venue"].sort_values().to_frame()
venues_count_rank[venues_count_rank["Venue"] < 10]

,Venue
District,
Pakal,5
Asemrowo,6
Kenjeran,7
Benowo,8
Semampir,8


The food venues are not spread evenly. Some districts have so many venues while other districts barely have any venues. We will use a minimum of 5 venues to cluster the districts later.

### The most popular venue category

In [185]:
venues["Venue Category"].value_counts().to_frame().head(10)

,Venue Category
Indonesian Restaurant,224
Food Truck,98
Café,92
Chinese Restaurant,82
Noodle House,65
Asian Restaurant,61
Bakery,52
Indonesian Meatball Place,41
Restaurant,39
Food Court,38


Unsurprisingly, Indonesian Restaurant tops the list with over 200+ venues listed. Followed by food truck and cafe, which is common in metropolitan city like Surabaya.

Chinese restaurant is sitting on #4, considering that Surabaya is home to many Chinese-Indonesian descendants.

### The most popular venue category in each district

In [186]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot['District'] = venues['District']
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]

# venues_onehot.head()

In [187]:
districts_grouped = venues_onehot.groupby('District').mean().reset_index()
districts_grouped.head()

,District,American Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gluten-free Restaurant,Halal Restaurant,Hot Dog Joint,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Korean Restaurant,Malay Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Noodle House,Padangnese Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Salad Place,Satay Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Snack Place,Soup Place,Steakhouse,Sundanese Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant
0,Asemrowo,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.000000,0.000000,0.166667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,Benowo,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.125000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.125000,0.000000,0.0,0.0,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Bubutan,0.017857,0.017857,0.0,0.000000,0.000000,0.017857,0.0,0.000000,0.017857,0.017857,0.0,0.0,0.000000,0.035714,0.000000,0.142857,0.0,0.0,0.0,0.035714,0.0,0.017857,0.000000,0.000000,0.0,0.000000,0.017857,0.0,0.071429,0.017857,0.053571,0.0,0.0,0.0,0.0,0.0,0.071429,0.160714,0.000000,0.000000,0.017857,0.017857,0.0,0.0,0.0,0.089286,0.000000,0.0,0.0,0.035714,0.0,0.000000,0.017857,0.017857,0.035714,0.053571,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bulak,0.000000,0.083333,0.0,0.041667,0.000000,0.041667,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.041667,0.000000,0.041667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.041667,0.0,0.166667,0.000000,0.0,0.333333,0.000000,0.041667,0.0,0.0,0.0,0.0,0.0,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Dukuh Pakis,0.000000,0.012658,0.0,0.012658,0.012658,0.012658,0.0,0.012658,0.000000,0.000000,0.0,0.0,0.000000,0.025316,0.012658,0.164557,0.0,0.0,0.0,0.000000,0.0,0.000000,0.037975,0.012658,0.0,0.000000,0.012658,0.0,0.000000,0.000000,0.025316,0.0,0.0,0.0,0.0,0.0,0.025316,0.227848,0.012658,0.063291,0.000000,0.037975,0.0,0.0,0.0,0.025316,0.012658,0.0,0.0,0.037975,0.0,0.000000,0.088608,0.000000,0.012658,0.037975,0.050633,0.0,0.012658,0.0,0.0,0.0,0.0,0.0


In [188]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

columns = ['District']
for ind in np.arange(num_top_venues):
    columns.append('#{} Most Common Venue'.format(ind+1))

venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['District'] = districts_grouped['District']

for ind in np.arange(districts_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(districts_grouped.iloc[ind, :], num_top_venues)

venues_sorted

,District,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue
0,Asemrowo,Seafood Restaurant,Chinese Restaurant,Food Truck,Cafeteria,Indonesian Restaurant
1,Benowo,Asian Restaurant,Indonesian Restaurant,Café,Restaurant,Fast Food Restaurant
2,Bubutan,Indonesian Restaurant,Chinese Restaurant,Noodle House,Food Truck,Indonesian Meatball Place
3,Bulak,Food Truck,Food,Asian Restaurant,Fried Chicken Joint,Seafood Restaurant
4,Dukuh Pakis,Indonesian Restaurant,Chinese Restaurant,Seafood Restaurant,Japanese Restaurant,Steakhouse
5,Gayungan,Indonesian Restaurant,Café,Food Truck,Restaurant,Chinese Restaurant
6,Genteng,Indonesian Restaurant,Japanese Restaurant,Café,Soup Place,Noodle House
7,Gubeng,Indonesian Restaurant,Bakery,Chinese Restaurant,Indonesian Meatball Place,Café
8,Gunung Anyar,Café,Asian Restaurant,Food Truck,Indonesian Meatball Place,Bakery
9,Jambangan,Food Truck,Indonesian Restaurant,Café,Food Court,Breakfast Spot


# Clustering the districts

### Making clusters of districts using K-Means

In [189]:
kclusters = 5
district_clustering = districts_grouped.drop("District", 1)

kmeans = KMeans(n_clusters=kclusters, random_state=99).fit(district_clustering)

kmeans.labels_

array([3, 1, 3, 4, 3, 1, 3, 3, 0, 0, 1, 0, 1, 0, 3, 0, 2, 4, 1, 3, 3, 0,
       1, 3, 1, 3, 3, 1, 0, 3, 3], dtype=int32)

In [190]:
districts_grouped_clustered = districts_grouped.copy()

districts_grouped_clustered.insert(0, "Cluster Labels", kmeans.labels_)
districts_grouped_clustered.head()

,Cluster Labels,District,American Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,Bistro,Breakfast Spot,Buffet,Burger Joint,Burrito Place,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gluten-free Restaurant,Halal Restaurant,Hot Dog Joint,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Korean Restaurant,Malay Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Noodle House,Padangnese Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Salad Place,Satay Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Snack Place,Soup Place,Steakhouse,Sundanese Restaurant,Sushi Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant
0,3,Asemrowo,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.000000,0.000000,0.166667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,1,Benowo,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.125000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.125000,0.000000,0.0,0.0,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,3,Bubutan,0.017857,0.017857,0.0,0.000000,0.000000,0.017857,0.0,0.000000,0.017857,0.017857,0.0,0.0,0.000000,0.035714,0.000000,0.142857,0.0,0.0,0.0,0.035714,0.0,0.017857,0.000000,0.000000,0.0,0.000000,0.017857,0.0,0.071429,0.017857,0.053571,0.0,0.0,0.0,0.0,0.0,0.071429,0.160714,0.000000,0.000000,0.017857,0.017857,0.0,0.0,0.0,0.089286,0.000000,0.0,0.0,0.035714,0.0,0.000000,0.017857,0.017857,0.035714,0.053571,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,4,Bulak,0.000000,0.083333,0.0,0.041667,0.000000,0.041667,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.041667,0.000000,0.041667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.041667,0.0,0.166667,0.000000,0.0,0.333333,0.000000,0.041667,0.0,0.0,0.0,0.0,0.0,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,3,Dukuh Pakis,0.000000,0.012658,0.0,0.012658,0.012658,0.012658,0.0,0.012658,0.000000,0.000000,0.0,0.0,0.000000,0.025316,0.012658,0.164557,0.0,0.0,0.0,0.000000,0.0,0.000000,0.037975,0.012658,0.0,0.000000,0.012658,0.0,0.000000,0.000000,0.025316,0.0,0.0,0.0,0.0,0.0,0.025316,0.227848,0.012658,0.063291,0.000000,0.037975,0.0,0.0,0.0,0.025316,0.012658,0.0,0.0,0.037975,0.0,0.000000,0.088608,0.000000,0.012658,0.037975,0.050633,0.0,0.012658,0.0,0.0,0.0,0.0,0.0


### Visualization of cluster spread

In [191]:
venues_sorted_clustered = venues_sorted.copy()
venues_sorted_clustered.insert(0, 'Cluster Labels', kmeans.labels_)
venues_sorted_clustered = pd.merge(venues_sorted_clustered, df, how='left', left_on=['District'], right_on=['name'])

In [192]:
venues_sorted_clustered.head()

,Cluster Labels,District,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,name,lat,lon
0,3,Asemrowo,Seafood Restaurant,Chinese Restaurant,Food Truck,Cafeteria,Indonesian Restaurant,Asemrowo,-7.237168,112.690060
1,1,Benowo,Asian Restaurant,Indonesian Restaurant,Café,Restaurant,Fast Food Restaurant,Benowo,-7.224294,112.645941
2,3,Bubutan,Indonesian Restaurant,Chinese Restaurant,Noodle House,Food Truck,Indonesian Meatball Place,Bubutan,-7.248752,112.727749
3,4,Bulak,Food Truck,Food,Asian Restaurant,Fried Chicken Joint,Seafood Restaurant,Bulak,-7.236022,112.788849
4,3,Dukuh Pakis,Indonesian Restaurant,Chinese Restaurant,Seafood Restaurant,Japanese Restaurant,Steakhouse,Dukuh Pakis,-7.291921,112.699990


In [193]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

map_out = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lon, poi, cluster in zip(venues_sorted_clustered['lat'], venues_sorted_clustered['lon'], venues_sorted_clustered['District'], venues_sorted_clustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7
    ).add_to(map_out)
       
map_out

On the map above, we can see that district's location and venue count number plays a big role to clustering. For example, Pakal district is the only district with the cluster label 2, and it is the district with the least venues and the most distant from the city center.

# Exploring each district clusters

In [194]:
def show_cluster_map(cluster_number=-1):
    map_df = venues_sorted_clustered
    f = None
    map_out = folium.Map(location=[latitude, longitude], zoom_start=11)
    if cluster_number != -1:
        map_df = venues_sorted_clustered[venues_sorted_clustered["Cluster Labels"] == cluster_number]
        f = folium.Figure(width=600, height=300)
        map_out = folium.Map(location=[latitude, longitude], zoom_start=11).add_to(f)
    

    for lat, lon, poi, cluster in zip(map_df['lat'], map_df['lon'], map_df['District'], map_df['Cluster Labels']):
        label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7
        ).add_to(map_out)
        
    return map_out

## Cluster 0

### Explanation: The Cafe and Food Truck Cluster

This cluster is located in the suburbs and dominated by food trucks and cafes.

In [195]:
venues_sorted_clustered[venues_sorted_clustered["Cluster Labels"] == 0]

,Cluster Labels,District,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,name,lat,lon
8,0,Gunung Anyar,Café,Asian Restaurant,Food Truck,Indonesian Meatball Place,Bakery,Gunung Anyar,-7.337686,112.795866
9,0,Jambangan,Food Truck,Indonesian Restaurant,Café,Food Court,Breakfast Spot,Jambangan,-7.323110,112.715666
11,0,Kenjeran,Fast Food Restaurant,Indonesian Meatball Place,Noodle House,Diner,Fried Chicken Joint,Kenjeran,-7.217600,112.768674
13,0,Lakarsantri,Café,Indonesian Restaurant,Soup Place,Food Truck,Restaurant,Lakarsantri,-7.308985,112.655482
15,0,Pabean Cantian,Café,Fast Food Restaurant,Food Truck,Indonesian Restaurant,Padangnese Restaurant,Pabean Cantian,-7.219347,112.733570
21,0,Simokerto,Indonesian Restaurant,Food Court,Food Truck,Noodle House,Snack Place,Simokerto,-7.239466,112.753292
28,0,Wiyung,Food Truck,Café,Pizza Place,Diner,Indonesian Restaurant,Wiyung,-7.312563,112.688257


In [196]:
show_cluster_map(0)

## Cluster 1

### Explanation: The Classic Suburban Cluster

This cluster is dominated by various restaurant types, but mostly local. We will call it the classic suburban cluster.

In [197]:
venues_sorted_clustered[venues_sorted_clustered["Cluster Labels"] == 1]

,Cluster Labels,District,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,name,lat,lon
1,1,Benowo,Asian Restaurant,Indonesian Restaurant,Café,Restaurant,Fast Food Restaurant,Benowo,-7.224294,112.645941
5,1,Gayungan,Indonesian Restaurant,Café,Food Truck,Restaurant,Chinese Restaurant,Gayungan,-7.329742,112.726671
10,1,Karang Pilang,Asian Restaurant,Indonesian Restaurant,Food Truck,Diner,Seafood Restaurant,Karang Pilang,-7.335119,112.688473
12,1,Krembangan,Indonesian Restaurant,Noodle House,American Restaurant,Soup Place,Donut Shop,Krembangan,-7.228817,112.721646
18,1,Sambikerep,Indonesian Restaurant,Café,Asian Restaurant,Chinese Restaurant,Sushi Restaurant,Sambikerep,-7.276603,112.653174
22,1,Sukolilo,Indonesian Restaurant,Asian Restaurant,Café,Chinese Restaurant,Burger Joint,Sukolilo,-7.291925,112.806840
24,1,Tambaksari,Indonesian Restaurant,Food Truck,Noodle House,Asian Restaurant,Café,Tambaksari,-7.250825,112.767967
27,1,Tenggilis Mejoyo,Indonesian Restaurant,Food Truck,Chinese Restaurant,Café,Restaurant,Tenggilis Mejoyo,-7.324517,112.754005


In [198]:
show_cluster_map(1)

## Cluster 2

### Explanation: The Pakal Cluster

This cluster only contains Pakal district. The district's location is very far from the city center and the venue choice is also strange relative to other clusters, dominated by Western and fast food restaurants. This district might be an outlier.

In [199]:
venues_sorted_clustered[venues_sorted_clustered["Cluster Labels"] == 2]

,Cluster Labels,District,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,name,lat,lon
16,2,Pakal,Fast Food Restaurant,Food,Burger Joint,Noodle House,Asian Restaurant,Pakal,-7.22906,112.616625


In [200]:
show_cluster_map(2)

## Cluster 3

### Explanation: The Downtown Cuisine Cluster

This cluster has the most districts and most of them are located in the city center, hence the most popular venue types are restaurants, foreign cuisine restaurants, cafes, foodcourts, and other "modern metropolitan" food palace.

In [201]:
venues_sorted_clustered[venues_sorted_clustered["Cluster Labels"] == 3]

,Cluster Labels,District,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,name,lat,lon
0,3,Asemrowo,Seafood Restaurant,Chinese Restaurant,Food Truck,Cafeteria,Indonesian Restaurant,Asemrowo,-7.237168,112.690060
2,3,Bubutan,Indonesian Restaurant,Chinese Restaurant,Noodle House,Food Truck,Indonesian Meatball Place,Bubutan,-7.248752,112.727749
4,3,Dukuh Pakis,Indonesian Restaurant,Chinese Restaurant,Seafood Restaurant,Japanese Restaurant,Steakhouse,Dukuh Pakis,-7.291921,112.699990
6,3,Genteng,Indonesian Restaurant,Japanese Restaurant,Café,Soup Place,Noodle House,Genteng,-7.258943,112.744854
7,3,Gubeng,Indonesian Restaurant,Bakery,Chinese Restaurant,Indonesian Meatball Place,Café,Gubeng,-7.279692,112.759391
14,3,Mulyorejo,Indonesian Restaurant,Chinese Restaurant,Japanese Restaurant,Noodle House,Café,Mulyorejo,-7.267390,112.797545
19,3,Sawahan,Indonesian Restaurant,Noodle House,Chinese Restaurant,Café,Fried Chicken Joint,Sawahan,-7.273047,112.721540
20,3,Semampir,Indonesian Restaurant,Fast Food Restaurant,Café,Cafeteria,Seafood Restaurant,Semampir,-7.213868,112.748414
23,3,Sukomanunggal,Indonesian Restaurant,Café,Asian Restaurant,Seafood Restaurant,Chinese Restaurant,Sukomanunggal,-7.268823,112.699726
25,3,Tandes,Food Court,Indonesian Restaurant,Fried Chicken Joint,Noodle House,Café,Tandes,-7.255170,112.671588


In [202]:
show_cluster_map(3)

## Cluster 4

### Explanation: The Food Trucks Cluster

This cluster is dominated by food trucks. Hence we will name it the food truck cluster.

In [203]:
venues_sorted_clustered[venues_sorted_clustered["Cluster Labels"] == 4]

,Cluster Labels,District,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,name,lat,lon
3,4,Bulak,Food Truck,Food,Asian Restaurant,Fried Chicken Joint,Seafood Restaurant,Bulak,-7.236022,112.788849
17,4,Rungkut,Food Truck,Indonesian Restaurant,Asian Restaurant,Bagel Shop,Indonesian Meatball Place,Rungkut,-7.319093,112.802315


In [204]:
show_cluster_map(4)

# The Final Map of Surabaya's District's Food Choice

In [205]:
show_cluster_map()